__Tips: Please install Bio before running by:__
pip install biopython

In [69]:
# python3 genome_simulation.py --ini='bacteria.ini' --data='Escherichia_coli_DH10B.fasta'
from configparser import ConfigParser
import optparse
import random, os, sys

import logging
from datetime import datetime
import numpy as np
import calendar;
import time;
from Bio import SeqIO 
from decimal import *

#### Step 0. Function definition:
__A. parse_file(filename, formatname, length):__ Return the sequencing from file with specified format and length.  
__B. mutate_prob(target_bases_list, prob):__ Mutate the base by picking from bases list with probability.  
__C. mutation_simulate(ori_seq):__ Mutate the original sequencing by probability/flipping and return the mutated sequencing, and how many bases have been mutated.

In [70]:
def parse_file(filename, formatname, length): 
    '''
        filename: file path
        formatname: 'fasta', 'abi', 'genbank or gb' etc...
        length: top number for snipping
    '''
    with open(filename, "rU") as handle:
        for record in SeqIO.parse(handle, formatname) :
            if len(record.seq)>length:
                base_top = record.seq[:length]
        return record, base_top

In [71]:
def mutate_prob(target_bases_list, prob1): 
    picked_base = np.random.choice(target_bases_list, 1, prob1) #Pick 1 from target list based on prob1.
    return picked_base

In [72]:
def mutation_simulate(ori_seq):
    count=0
    mutated_seq=[]

    for ori_base in ori_seq:
        flip = np.random.uniform(0, 1)
        #print('flip on ', ori_base, flip)
        
        lambda_list = list(conf_lamba)
        picked_lamba = np.random.choice(lambda_list, 1)
    
        # For C base: transit to A or G by flip probability (if flip < lamda picked randomly from configuration lambda list.)
        if (ori_base == 'C') and (flip <= picked_lamba): #C-> G or A
            mutated_base = mutate_prob(['A','G'], flip)  #Pick 1 from target list based on flip.
            count+=1
            if mutated_base:
                #print ('(Mutated', ori_base, ' to: ', mutated_base, ' flip: ', flip, 'lambda: ', picked_lamba, ')')
                mutated_seq.append(mutated_base)   
            else:
                mutated_seq.append(ori_base)
                
        # For G base: transit to C or T by flip probability (if flip < lamda picked randomly from configuration lambda list.)       
        elif (ori_base == 'G') and (flip <= picked_lamba): #G-> C or T
            mutated_base = mutate_prob(['C','T'], flip)  #Pick 1 from target list based on flip.
            count+=1
            if mutated_base:
                #print ('(Mutated', ori_base, ' to: ', mutated_base, ' flip: ', flip, 'lambda: ', picked_lamba, ')')
                mutated_seq.append(mutated_base)   
            else:
                mutated_seq.append(ori_base)            
        else:
            mutated_seq.append(ori_base)

    return mutated_seq, count

#### Step 1. Please configure these parameters accordingly before running:
- ALL parameters in configuration will be kept in the format of 'conf_XXXXX' for easily recognization.

In [73]:
#[lamba]
# conf_lamba_min/max: It is on a per generation basis ... update mutation rate to different values within the range. 
# conf_lamba_min/max:Start by one generation the min rate then the next the max rate then the min rate etc
# conf_lamba_min=0.1 
# conf_lamba_max=0.5

# conf_lamba: can have multiple lambas
conf_lamba=0.2,0.1,0.3
conf_generation=7

#[mutation]
# run_times: How many mutated seqs will be generated.
conf_run_times=4 
conf_numrows=1

#[fasta]
# conf_filepath: The path of fasta file.
conf_filepath = '../Escherichia_coli_DH10B.fasta'
# conf_top_num: How many bases could be kept from fasta file.
conf_top_num = 40 
# conf_generated_file_format: specify the format will be generated with: '.nex','.txt' etc.
conf_generated_file_format = '.txt'


#### Step 2: Get the specified length of the sequences from the specified file.

In [74]:
# 0. FAKE seq for testing:
# ori_seq='AGCTACTGGC'

In [75]:
# Get the trimmed seq string from specified file.
record, base_top = parse_file(conf_filepath, "fasta", int(conf_top_num))
ori_seq = str(base_top)
print(ori_seq)

AGCTTTTCATTCTGACTGCAACGGGCAATATGTCTCTGTG


/Users/sikedong/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: 'U' mode is deprecated
  import sys


#### Step 3: Generate the mutated sequences with specified times and write these mutated sequeces to file with specified format.
- In this case, write the mutated sequeces in .nex file.  
- The file will be named with timestamp.  
- e.g.: __Generated file: GBAC_<Generation\><mutation_rate\>Seq__  
GBAC_7_0.2 AGCTTTTCATTCTGACTGAAACGGGCAATATGTCTCTGTG  
GBAC_7_0.2 AGATTTTCATTCTGACTGCAACGGTCAATATGTCTCTGTG  
GBAC_7_0.2 AGCTTTTCATTCTGACTGCAACGCGCAATATTTCTATCTC  
GBAC_7_0.2 AGATTTTCATTCTGACTTCAACGCGAAATATGTCTCTGTG  
GBAC_7_0.2 AGCTTTTCATTCTGACTGCAACGGGCAATATGTCTCTGTG  

In [76]:
# Generate file name with timestamp.
timestamp = calendar.timegm(time.gmtime())
print(timestamp)
mut_seq_nexfilepath = "file_generated\\"+"SimSeqFile_"+ str(timestamp) + conf_generated_file_format

1565734419


In [77]:
def compare_seq(seq1, seq2):
    diff_seq = ''
    diff_index = ''
    diff=''
    
    if len(seq1) == len(seq2):
        for i in range(0, len(seq1)):
            if seq1[i]!=seq2[i]:
                diff_seq+=seq2[i]
                diff_index+=str(i)+','
        diff = diff_seq +' '+ diff_index
        
    else:
        return null
    
    return diff

In [78]:
#Sike difference marker
def dna_seq_compare(ori_seq, mutated_seq):
    len1 = len(ori_seq)
    len2 = len(mutated_seq)
    mismatches = []
    for pos in range (0, min(len1, len2)):
        if ori_seq[pos]!= mutated_seq[pos]:
            mismatches.append("|")
        else:
            mismatches.append(" ")
    mismatches = "".join(mismatches)

    return mismatches+'\n'+ mutated_seq

In [79]:
#Sike compare two sequences, equal = 0, different = 1
def dna_seq_compare_equality(seq1, seq2):
    len1 = len(seq1)
    len2 = len(seq2)
    result = []
    if len(seq1) == len(seq2):
        for i in range(0, min(len1, len2)):
            if seq1[i]!=seq2[i]:
                result.append("1")
            else:
                result.append("0")
    result = "".join(result)

    return result +'\n'

In [80]:
multi_mutated_seq=''
diff_str=''
mark_str = ''
compare_ori_mut='Result:'+'\n'
multi_mutated_seq='Original_Sequencing: ' + ori_seq+ '\n'

for run_time in range(conf_run_times):
    # print('\nMutating...', run_time+1, 'times: ')
    mutated_seq_file_list,count = mutation_simulate(ori_seq) #Tuple return: mutation_simulate(): mutated_seq, count. Get mutated_seq.
        
    #seq_num = 'GBAC_' + str(conf_generation)+'_'+ str(conf_lamba)
    mutated_seq= ''.join([''.join(base) for base in mutated_seq_file_list])
    #multi_mutated_seq += seq_num +':'+ mutated_seq+'\n'
    
    diff_str += str(run_time+1) +' time(s) mutation: '+ compare_seq(ori_seq, mutated_seq)+'\n'
    
    #Sike difference marker, compare two sequences, equal = 0, different = 1
    str_diff_result = ori_seq + '\n'+ dna_seq_compare(ori_seq, mutated_seq) + '\n' 
    print(str_diff_result)
    mark_str += str_diff_result + '\n'
    compare_result = dna_seq_compare_equality(ori_seq, mutated_seq) + '\n' 
    print(compare_result)
    compare_ori_mut += compare_result + '\n'
    
#print(mutated_seq)
#print(multi_mutated_seq)
print(diff_str)
print(compare_ori_mut)

AGCTTTTCATTCTGACTGCAACGGGCAATATGTCTCTGTG
                       |       | |      
AGCTTTTCATTCTGACTGCAACGTGCAATATTTGTCTGTG

0000000000000000000000010000000101000000


AGCTTTTCATTCTGACTGCAACGGGCAATATGTCTCTGTG
 |         |   | ||    ||      |        
ATCTTTTCATTGTGAGTTAAACGTTCAATATCTCTCTGTG

0100000000010001011000011000000100000000


AGCTTTTCATTCTGACTGCAACGGGCAATATGTCTCTGTG
           |           ||            |  
AGCTTTTCATTGTGACTGCAACGCCCAATATGTCTCTTTG

0000000000010000000000011000000000000100


AGCTTTTCATTCTGACTGCAACGGGCAATATGTCTCTGTG
                               |        
AGCTTTTCATTCTGACTGCAACGGGCAATATCTCTCTGTG

0000000000000000000000000000000100000000


1 time(s) mutation: TTG 23,31,33,
2 time(s) mutation: TGGTATTC 1,11,15,17,18,23,24,31,
3 time(s) mutation: GCCT 11,23,24,37,
4 time(s) mutation: C 31,

Result:
0000000000000000000000010000000101000000


0100000000010001011000011000000100000000


0000000000010000000000011000000000000100


0000000000000000000000000000000100000000




In [81]:
def write_file(filepath, content):
    with open(filepath, "w") as file:
        file.write(content)
        print('Done. Wrote to: ', filepath)

In [82]:
write_file(mut_seq_nexfilepath, diff_str)

Done. Wrote to:  file_generated\SimSeqFile_1565734419.txt


In [83]:
diff_filepath = "file_generated\\" +"Diff_Ori_Mut_" + str(timestamp) + conf_generated_file_format
write_file(diff_filepath, mark_str)

Done. Wrote to:  file_generated\Diff_Ori_Mut_1565734419.txt


In [84]:
compare_filepath = "file_generated\\" +"Comp_Ori_Mut_" + str(timestamp) + conf_generated_file_format
write_file(compare_filepath, compare_ori_mut)

Done. Wrote to:  file_generated\Comp_Ori_Mut_1565734419.txt


In [108]:
#Sike - dendrogram
import pandas as pd
import scipy 
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial import distance
from scipy.spatial.distance import pdist
import matplotlib.pyplot as plt

In [131]:
datafile = 'file_generated\Comp_Ori_Mut_1565734419.txt'
df = pd.read_csv(datafile, sep='\t')
df = df.set_index('Result:')
df

""
Result:
0000000000000000000000010000000101000000
0100000000010001011000011000000100000000
0000000000010000000000011000000000000100
0000000000000000000000000000000100000000


In [143]:
df.dtypes

Series([], dtype: object)

In [144]:
Y=pdist(df_gene,'hamming')
Z=linkage(Y,'single')
plt.title('Gene Mutation')
plt.xlabel('Gene Diifference')
plt.ylabel('distance')
dendrogram(Z,labels=df.index, leaf_rotation=90)
plt.show()

ValueError: The condensed distance matrix must contain only finite values.